#Задание
* Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
* Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
* Попробуйте добавить +1 рекуррентный слой в encoder и decoder
* Попробуйте заменить GRU ячейки на lstm-ячейки

In [ ]:
%matplotlib inline
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-06-14 11:12:20--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15460248 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.74M  51.4MB/s    in 0.3s    

2023-06-14 11:12:21 (51.4 MB/s) - ‘rus-eng.zip’ saved [15460248/15460248]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [ ]:
!tail rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+а-яА-Я", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('rus.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    for p in pairs:
        p.pop()

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)

    else:

        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):

    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 467119 sentence pairs
Trimmed to 4269 sentence pairs
Counting words...
Counted words:
rus 3913
eng 2071
['он менее терпелив, чем его брат .', 'he is less patient than his brother .']


The Encoder
-----------

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 31s (- 21m 25s) (5000 6%) 3.0835
3m 5s (- 20m 4s) (10000 13%) 2.4664
4m 40s (- 18m 41s) (15000 20%) 1.9849
6m 15s (- 17m 13s) (20000 26%) 1.5713
7m 50s (- 15m 40s) (25000 33%) 1.2122
9m 24s (- 14m 6s) (30000 40%) 0.9321
10m 58s (- 12m 32s) (35000 46%) 0.6690
12m 32s (- 10m 58s) (40000 53%) 0.5174
14m 8s (- 9m 25s) (45000 60%) 0.3800
15m 43s (- 7m 51s) (50000 66%) 0.2834
17m 17s (- 6m 17s) (55000 73%) 0.1848
18m 52s (- 4m 43s) (60000 80%) 0.1363
20m 27s (- 3m 8s) (65000 86%) 0.1109
22m 2s (- 1m 34s) (70000 93%) 0.0819
23m 36s (- 0m 0s) (75000 100%) 0.0582


In [ ]:
evaluateRandomly(encoder1, decoder1, 25)

> у него аллергия на пыль .
= he is allergic to dust .
< he is allergic to dust . <EOS>

> она моя одноклассница .
= she is my classmate .
< she is my classmate . <EOS>

> я разведен .
= i am divorced .
< i am divorced . <EOS>

> я боюсь смерти .
= i am afraid of dying .
< i am afraid of death . <EOS>

> он, что называется, человек, которыи сделал себя сам .
= he is what is called a self-made man .
< he is what we call a self-made man . <EOS>

> я боюсь собак .
= i am afraid of dogs .
< i am afraid of dogs . <EOS>

> она говорит по-португальски .
= she speaks portuguese .
< she speaks portuguese . <EOS>

> мне стыдно .
= i am ashamed .
< i am ashamed . <EOS>

> она мне не подходит .
= she is no match for me .
< she is no match for me . <EOS>

> его часто не бывает в школе .
= he is often absent from school .
< he is often absent from school . <EOS>

> он занят подготовкои к поездке .
= he is busy preparing for the trip .
< he is busy preparing for the trip . <EOS>

> он мои брат .
= he

Добавим в encoder и decoder возможность добавлять рекурретный слой и менять rnn GRU/LSTM

In [ ]:
class EncoderRNN1(nn.Module):
    def __init__(self, input_size, hidden_size, num_rnn = 1, rnnClass = "GRU"):
        super(EncoderRNN1, self).__init__()
        self.hidden_size = hidden_size
        self.num_rnn = num_rnn
        self.embedding = nn.Embedding(input_size, hidden_size)

        if rnnClass == "GRU":
          self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnnClass == "LSTM":
          self.rnn = nn.LSTM(hidden_size, hidden_size)

        if self.num_rnn == 2:
          if rnnClass == "GRU":
              self.rnn_2 = nn.GRU(hidden_size, hidden_size)
          elif rnnClass == "LSTM":
              self.rnn_2 = nn.LSTM(hidden_size, hidden_size)


    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)

        if self.num_rnn == 2:
          output, hidden = self.rnn_2(output, hidden)

        return output, hidden

    def initHidden(self):
      if isinstance(self.rnn, nn.LSTM):
          return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))
      return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN1(nn.Module):
    def __init__(self, hidden_size, output_size, num_rnn = 1, rnnClass = "GRU"):
        super(DecoderRNN1, self).__init__()
        self.hidden_size = hidden_size
        self.num_rnn = num_rnn
        self.embedding = nn.Embedding(output_size, hidden_size)

        if rnnClass == "GRU":
          self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnnClass == "LSTM":
          self.rnn = nn.LSTM(hidden_size, hidden_size)
        if self.num_rnn == 2:
           if rnnClass == "GRU":
               self.rnn_2 = nn.GRU(hidden_size, hidden_size)
           elif rnnClass == "LSTM":
               self.rnn_2 = nn.LSTM(hidden_size, hidden_size)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)

        if self.num_rnn == 2:
          output, hidden = self.rnn_2(output, hidden)

        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)



#GRU +1 рекуррентный слой в encoder и decoder

In [ ]:
hidden_size = 256
encoder2 = EncoderRNN1(input_lang.n_words, hidden_size, num_rnn=2).to(device)
decoder2 = DecoderRNN1(hidden_size, output_lang.n_words, num_rnn=2).to(device)

trainIters(encoder2, decoder2, 75000, print_every=5000)

2m 18s (- 32m 20s) (5000 6%) 3.3312
4m 36s (- 29m 57s) (10000 13%) 2.8534
6m 54s (- 27m 38s) (15000 20%) 2.6345
9m 17s (- 25m 32s) (20000 26%) 2.4974
11m 38s (- 23m 16s) (25000 33%) 2.2613
14m 2s (- 21m 4s) (30000 40%) 2.0063
16m 25s (- 18m 45s) (35000 46%) 1.8461
18m 46s (- 16m 25s) (40000 53%) 1.5996
21m 9s (- 14m 6s) (45000 60%) 1.4149
23m 34s (- 11m 47s) (50000 66%) 1.1855
26m 3s (- 9m 28s) (55000 73%) 0.9919
28m 30s (- 7m 7s) (60000 80%) 0.8329
30m 57s (- 4m 45s) (65000 86%) 0.6436
33m 21s (- 2m 22s) (70000 93%) 0.4969
35m 49s (- 0m 0s) (75000 100%) 0.4132


In [ ]:
evaluateRandomly(encoder2, decoder2, 25)

> они обессилены .
= they are exhausted .
< they are exhausted . <EOS>

> они неразлучны .
= they are inseparable .
< they are approaching . <EOS>

> мы с тобои примерно ровесники .
= you are about my age .
< you are about my age . <EOS>

> он студент университета .
= he is a university student .
< he is a university student . <EOS>

> он хитрая лиса .
= he is a sly fox .
< he is a sly fox . <EOS>

> он слишком осторожен, чтобы попробовать что-либо новое .
= he is too cautious to try anything new .
< he is too cautious to try anything new . <EOS>

> она переплыла реку .
= she swam across the river .
< she swam across the river . <EOS>

> мы извиняемся за неудобства .
= we are sorry for the inconvenience .
< we are going for peace . <EOS>

> мы сотрудничаем со многими странами .
= we are doing business with many countries .
< we are doing with many with their . . <EOS>

> он играет в своеи комнате .
= he is playing in his room .
< he is playing in his room . <EOS>

> я так измучена !
= 


#LSTM

In [ ]:
hidden_size = 256
encoder3 = EncoderRNN1(input_lang.n_words, hidden_size, rnnClass = "LSTM").to(device)
decoder3 = DecoderRNN1(hidden_size, output_lang.n_words, rnnClass = "LSTM").to(device)

trainIters(encoder3, decoder3, 75000, print_every=5000)

2m 11s (- 30m 35s) (5000 6%) 3.2284
4m 25s (- 28m 46s) (10000 13%) 2.7010
6m 40s (- 26m 43s) (15000 20%) 2.3382
8m 57s (- 24m 39s) (20000 26%) 1.9521
11m 12s (- 22m 25s) (25000 33%) 1.6087
13m 29s (- 20m 14s) (30000 40%) 1.3167
15m 45s (- 18m 0s) (35000 46%) 1.0836
18m 2s (- 15m 47s) (40000 53%) 0.8374
20m 19s (- 13m 33s) (45000 60%) 0.6652
22m 34s (- 11m 17s) (50000 66%) 0.5212
24m 50s (- 9m 1s) (55000 73%) 0.3755
27m 7s (- 6m 46s) (60000 80%) 0.2835
29m 24s (- 4m 31s) (65000 86%) 0.2056
31m 41s (- 2m 15s) (70000 93%) 0.1698
33m 58s (- 0m 0s) (75000 100%) 0.1181


In [ ]:
evaluateRandomly(encoder3, decoder3, 25)

> я ему не подхожу .
= i am no match for him .
< i am no match for him . <EOS>

> мы очень рады вас видеть .
= we are happy to see you .
< we are happy to see you . <EOS>

> еи рано идти в школу .
= she is too young to go to school .
< she is too young to go to school . <EOS>

> он или пьян, или безумен .
= he is either drunk or mad .
< he is either drunk or mad . <EOS>

> он очень осторожныи водитель .
= he is a very careful driver .
< he is a very careful driver . <EOS>

> я из токио .
= i am from tokyo .
< i am from tokyo . <EOS>

> у него редко хорошее настроение .
= he is rarely in a good mood .
< he is rarely in a good mood . <EOS>

> я первыи музыкант в нашеи семье .
= i am the first musician in my family .
< i am the first musician in my family . <EOS>

> он откладывает деньги на поездку за границу .
= he is saving money for a trip abroad .
< he is saving money for a trip abroad . <EOS>

> я боюсь смерти .
= i am afraid of dying .
< i am afraid of death . <EOS>

> он вовсе не с


#LSTM +1 рекуррентный слой

In [ ]:
hidden_size = 256
encoder4 = EncoderRNN1(input_lang.n_words, hidden_size, num_rnn=2, rnnClass = "LSTM").to(device)
decoder4 = DecoderRNN1(hidden_size, output_lang.n_words, num_rnn=2, rnnClass = "LSTM").to(device)

trainIters(encoder4, decoder4, 75000, print_every=5000)

3m 38s (- 50m 54s) (5000 6%) 3.3987
7m 20s (- 47m 44s) (10000 13%) 3.1288
11m 4s (- 44m 18s) (15000 20%) 2.8548
14m 52s (- 40m 53s) (20000 26%) 2.6272
18m 38s (- 37m 17s) (25000 33%) 2.5342
22m 26s (- 33m 39s) (30000 40%) 2.4481
26m 13s (- 29m 58s) (35000 46%) 2.3344
29m 54s (- 26m 9s) (40000 53%) 2.2013
33m 37s (- 22m 25s) (45000 60%) 2.1008
37m 30s (- 18m 45s) (50000 66%) 1.9850
41m 17s (- 15m 0s) (55000 73%) 1.7948
45m 10s (- 11m 17s) (60000 80%) 1.6679
49m 3s (- 7m 32s) (65000 86%) 1.5526
52m 57s (- 3m 46s) (70000 93%) 1.3925
56m 52s (- 0m 0s) (75000 100%) 1.3022


In [ ]:
evaluateRandomly(encoder4, decoder4, 25)

> вы же не очень заняты ?
= you aren't very busy, are you ?
< you aren't very busy, are you ? <EOS>

> ты опоздал .
= you are too late .
< you are late . <EOS>

> они соседи .
= they are neighbors .
< they are hungry . <EOS>

> она так же красива, как и ее мать .
= she is as beautiful as her mother .
< she is as beautiful as as mother . <EOS>

> он слишком осторожен, чтобы попробовать что-либо новое .
= he is too cautious to try anything new .
< he is too to to go to . . <EOS>

> она шьет платье .
= she is sewing a dress .
< she is wearing a . . <EOS>

> нас разделяет только тихии океан .
= we are only separated by the pacific ocean .
< we are having of time, tom of diseases . <EOS>

> они не маленькие дети .
= they are not little children .
< they aren't not us . <EOS>

> она пользуется большои популярностью у мальчиков .
= she is very popular among the boys .
< she is dressed in white . <EOS>

> еи противна ее работа .
= she is disgusted with the job .
< she is living an unhappy life

Значение функции потерь:
* GRU 1 рекуррентный слой   loss = 0.0582
* GRU 2 рекуррентных слоя   loss = 0.4132
* LSTM 1 рекуррентный слой  loss = 0.1181
* LSTM 2 рекуррентных слоя  loss = 1.3022

2 слоя хуже чем один

LSTM с 2 рекуррентными слоями переводит плохо